## Segundo Experimento: Comparação Local do m² dos Bairros

In [ ]:
# Vamos pegar os 100 bairros com o maior número de imóveis na base
bairros_para_analise = itbi['Bairro'].value_counts().nlargest(100).index

print("Analisando as dinâmicas internas dos seguintes bairros:")
print(bairros_para_analise.tolist())
print("="*60)

In [ ]:
print("Iniciando Análise Focada por Bairro com comparação Nominal e Real...")
print("="*70)

for bairro_foco in bairros_para_analise:
    print(f"\n--- Análise Focada no Bairro: {bairro_foco} ---")
    
    # 1. Filtra o DataFrame para o bairro em questão
    df_bairro = itbi[itbi['Bairro'] == bairro_foco].copy()
    
    if len(df_bairro) < 50:
        print("Dados insuficientes para uma análise robusta. Pulando...")
        continue

    # 2. Calcula as médias de base PARA ESTE BAIRRO ESPECÍFICO
    media_bairro_nominal = df_bairro['valor_m2'].mean()
    media_bairro_real = df_bairro['valor_m2_corrigido'].mean()
    print(f"Média Nominal do Bairro: R$ {media_bairro_nominal:,.2f}")
    print(f"Média Real do Bairro: R$ {media_bairro_real:,.2f}\n")

    # 3. Executa a análise do pysubgroup
    target = ps.NumericTarget('valor_m2') # Alvo é o valor nominal
    quality_function = ps.StandardQFNumeric(a=0.5)
    
    include = ['Tipo Construtivo Preponderante', 'Ano Avaliacao', 'idade']
    ignore = list(set(list(itbi)) - set(include))
    search_space = ps.create_selectors(df_bairro, ignore=ignore)

    task = ps.SubgroupDiscoveryTask(
        df_bairro,
        target,
        search_space,
        result_set_size=5,
        depth=2,
        qf=quality_function
    )
    
    result = ps.BeamSearch().execute(task)
    df_results = result.to_dataframe()

    if df_results.empty:
        print("Nenhum padrão interessante encontrado para este bairro.")
        print("-" * 50)
        continue

    # --- INÍCIO DA ATUALIZAÇÃO NA IMPRESSÃO ---

    # 4. Imprime os resultados com as duas métricas de impacto
    for index, row in df_results.iterrows():
        subgrupo_obj = row['subgroup']
        
        # Filtra os dados do bairro para obter os dados apenas deste subgrupo
        dados_do_subgrupo = subgrupo_obj.covers(df_bairro)
        
        # Coleta as médias nominais e reais do subgrupo
        media_nominal_sg = row['mean_sg']
        media_real_sg = df_bairro[dados_do_subgrupo]['valor_m2_corrigido'].mean()
        
        # Calcula os dois impactos percentuais em relação às médias do BAIRRO
        impacto_nominal = ((media_nominal_sg / media_bairro_nominal) - 1) * 100
        impacto_real = ((media_real_sg / media_bairro_real) - 1) * 100
        
        # Imprime o "card" de resultado aprimorado
        print(f"Qualidade: {row['quality']:.2f} | Padrão Interno: {row['subgroup']}")
        print(f"   > Tamanho: {row['size_sg']:,.0f} imóveis")
        print(f"   > Média Nominal: R$ {media_nominal_sg:,.2f}  (Impacto: {impacto_nominal:+.2f}% vs. média NOMINAL do bairro)")
        print(f"   > Média Real   : R$ {media_real_sg:,.2f}  (Impacto: {impacto_real:+.2f}% vs. média REAL do bairro)")
        print("-" * 50)
